In [1]:
import os
import torch
import torch.nn as nn
from tqdm import tqdm
import pandas as pd
import csv
from torchvision import transforms
from PIL import Image
import shutil

class ImageComparisonModel(nn.Module):
    def __init__(self):
        super(ImageComparisonModel, self).__init__()
        
        # Stream 1 for the first image modality
        self.stream1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Stream 2 for the second image modality
        self.stream2 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Fusion and decision mechanism
        self.comparison = nn.Sequential(
            nn.Linear(16*16*256*2, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1),  # Binary classification
        )
        
    def forward(self, x1, x2):
        x1 = self.stream1(x1)
        x2 = self.stream2(x2)
        
        # Flatten the features from both streams
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        
        # Concatenate the features
        x = torch.cat((x1, x2), dim=1)
        
        # Pass through the comparison mechanism
        x = self.comparison(x)
        # Apply softmax to get probabilities
        # x = F.softmax(x, dim=1)
        return x

In [2]:
# 저장된 .pth 파일의 경로
saved_model_path = "Check_Model.pth"

# 저장된 모델 클래스의 인스턴스 생성
loaded_model = ImageComparisonModel()

# 저장된 모델의 가중치 불러오기
loaded_model.load_state_dict(torch.load(saved_model_path))

# 모델을 평가 모드로 설정
loaded_model.eval()

# CSV 파일 경로
csv_file_path = "ddpm_data/ddpm_processed.csv"

# 경로 설정
root_dir1 = 'ddpm_data/chemdraw_ddpm'
root_dir2 = 'ddpm_data/Sample_image_1'

# 폴더 내 모든 이미지 파일 경로 가져오기
image_files1 = [os.path.join(root_dir1, filename) for filename in os.listdir(root_dir1) if filename.endswith(('.png', '.jpg', '.jpeg'))]
image_files2 = [os.path.join(root_dir2, filename) for filename in os.listdir(root_dir2) if filename.endswith(('.png', '.jpg', '.jpeg'))]

# 이미지 파일을 모두 불러와서 리스트에 저장
images1 = [Image.open(img_path).convert("RGB") for img_path in image_files1]
images2 = [Image.open(img_path).convert("RGB") for img_path in image_files2]

# 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 이미지 크기를 64x64로 조정
    transforms.ToTensor(),         # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
])

preprocessed_images1 = [transform(img) for img in images1]
preprocessed_images2 = [transform(img) for img in images2]

# CSV 파일을 DataFrame으로 불러오기
df = pd.read_csv(csv_file_path)

# 모델을 CUDA 장치로 이동

# GPU를 사용할 수 있는지 확인합니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loaded_model.to(device)

# 분류 결과를 저장할 리스트
zero_label_images = []


In [3]:
# 분류 결과를 저장할 리스트
zero_label_images = []

# 결과를 저장할 리스트
results = []

# tqdm을 사용하여 진행 상황 표시
for img1_name, img1, preprocessed_img1 in zip(image_files1, tqdm(images1, desc="Processing images", unit="image"), preprocessed_images1):
    # img1과 대응되는 이미지2를 찾음
    img1_idx = image_files1.index(img1_name)  # img1의 인덱스
    img2_name, img2, preprocessed_img2 = image_files2[img1_idx], images2[img1_idx], preprocessed_images2[img1_idx]  # 대응되는 img2의 이름과 이미지, 전처리된 이미지

    # 전처리된 이미지를 모델 입력에 맞게 GPU로 이동
    preprocessed_img1_tensor = preprocessed_img1.unsqueeze(0).to(device)  
    preprocessed_img2_tensor = preprocessed_img2.unsqueeze(0).to(device)  

    # 이미지를 비교하여 예측 수행
    with torch.no_grad():
        output = loaded_model(preprocessed_img1_tensor, preprocessed_img2_tensor)
        similarity = torch.sigmoid(output).item()

    # 결과 저장
    results.append([img1_name, img2_name, similarity])
    
    # 예측 레이블이 0인 경우 해당 이미지의 인덱스를 리스트에 추가
    if similarity < 0.5:
        zero_label_images.append(img1_idx)

print("완료되었습니다.")

# 결과를 DataFrame으로 변환
df = pd.DataFrame(results, columns=['image1_name', 'image2_name', 'similarity'])

# zero_label_images를 사용하여 df 필터링
df_filtered = df.drop(zero_label_images)


Processing images:  98%|█████████▊| 39/40 [00:01<00:00, 30.42image/s]

완료되었습니다.


In [10]:
def copy_and_remove_images(destination_folder, image_names):
    """
    destination_folder: 이미지 파일을 복사할 대상 폴더 경로
    image_names: 복사 및 삭제할 이미지 파일 이름 리스트 (상대 경로 포함)
    """
    # 대상 폴더가 없는 경우 새로 생성
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for image_name in image_names:
        # 이미지 파일 경로
        source_path = image_name  # 이미지 파일의 상대 경로가 이미 포함되어 있음
        
        # 이미지 파일 복사
        shutil.copy(source_path, destination_folder)
        
        # 원본 이미지 파일 삭제
        os.remove(source_path)

In [8]:
zero_label_image_names

['ddpm_data/Sample_image_1/image_25.png',
 'ddpm_data/Sample_image_1/image_3.png',
 'ddpm_data/Sample_image_1/image_44.png',
 'ddpm_data/Sample_image_1/image_13.png',
 'ddpm_data/Sample_image_1/image_21.png',
 'ddpm_data/Sample_image_1/image_19.png',
 'ddpm_data/Sample_image_1/image_35.png',
 'ddpm_data/Sample_image_1/image_40.png',
 'ddpm_data/Sample_image_1/image_49.png',
 'ddpm_data/Sample_image_1/image_18.png',
 'ddpm_data/Sample_image_1/image_8.png',
 'ddpm_data/Sample_image_1/image_5.png',
 'ddpm_data/Sample_image_1/image_38.png',
 'ddpm_data/Sample_image_1/image_34.png',
 'ddpm_data/Sample_image_1/image_32.png',
 'ddpm_data/Sample_image_1/image_0.png',
 'ddpm_data/Sample_image_1/image_30.png',
 'ddpm_data/Sample_image_1/image_22.png',
 'ddpm_data/Sample_image_1/image_43.png']

In [11]:
# zero_label_images에 있는 이미지의 파일 이름 가져오기
zero_label_image_names = [image_files2[idx] for idx in zero_label_images]

# 상대 경로로 지정된 폴더 경로를 절대 경로로 변환

# 상대 경로로 지정된 폴더 경로
destination_folder = 'ddpm_data/ddpm_data_processed'  # 이미지 파일을 복사할 대상 폴더 이름

# 이미지를 복사하여 삭제된 상태로 저장
copy_and_remove_images(destination_folder, zero_label_image_names)

In [ ]:
# 필터링된 결과를 새로운 CSV 파일로 저장
output_filtered_csv_path = "filtered_image_similarity_results.csv"
df_filtered.to_csv(output_filtered_csv_path, index=False)

print(f"Filtered results saved to {output_filtered_csv_path}")